In [1]:
import numpy as np
import numpy.linalg as LA
import pandas as pd
import networkx as nx
from tqdm.notebook import tqdm
from surprise import SVD
from surprise import Dataset
from surprise import model_selection
from surprise import accuracy
from surprise import Reader
from surprise.prediction_algorithms.predictions import Prediction

In [2]:
class PageRankRecommender():
    
    def __init__(self, alpha, eps = None, max_iter = None, algo = 'sharp'):
        
        self.alpha = alpha
        self.beta = 2 * (1 - alpha) / alpha
        self.max_iter = max_iter
        self.eps = eps
        self.algo = algo
        
        self.predicted = {}
        self.possible_ratings = []
    
    def __build_graph__(self, data):
        G = nx.Graph()
        
        for user, movie, rating in data.all_ratings():
            user_node = ('u', int(data.to_raw_uid(user)))
            movie_node = ('m', int(data.to_raw_iid(movie)), rating)
            G.add_edge(user_node, movie_node)
        
        return G
    
    def __build_val_map__(self, data):
        return {(int(user), int(movie)) : float(rating) for (user, movie, rating) in data}
    
    def fit(self, data, val_data = None):
        logging = (val_data is not None)
        
        self.possible_ratings = sorted(set([rating for (_, _, rating) in data.all_ratings()]))
        self.mean_rating = data.global_mean
        
        G = self.__build_graph__(data)
        
        node_to_idx = dict(zip(G.nodes, range(len(G))))
        users = [node[1] for node in G.nodes if node[0] == 'u']
        movies = list(set([node[1] for node in G.nodes if node[0] == 'm']))
        
        if logging:
            val_map = self.__build_val_map__(val_data)
            MSE_sum = 0
            MSE_cnt = 0
        
        users_bar = tqdm(users)
        for user in users_bar:
            t = {node : 0 for node in G.nodes}
            t[('u', user)] = 1

            if self.algo == 'pagerank':
                p = nx.pagerank(G, personalization = t, alpha = self.alpha)
            else:
                p = self.__sharp_pagerank__(G, node_to_idx, t = t)

            for movie in movies:
                
                weighted_sum = sum([p[('m', movie, rating)] for rating in self.possible_ratings if ('m', movie, rating) in p])
                weighted_rating = sum([rating * p[('m', movie, rating)] for rating in self.possible_ratings if ('m', movie, rating) in p])
                        
                if weighted_sum == 0:
                    self.predicted[user, movie] = self.mean_rating
                else:
                    self.predicted[user, movie] = weighted_rating / weighted_sum
                
                if logging:
                    if (user, movie) in val_map:
                        MSE_sum += (val_map[user, movie] - self.predicted[user, movie]) ** 2
                        MSE_cnt += 1

            if logging and MSE_cnt > 0:
                users_bar.set_description(f'MSE = {MSE_sum / MSE_cnt :.5f} | Progress')
    
    def test(self, data):
        predictions = []
        
        for user, movie, rating in data:
            user = int(user)
            movie = int(movie)
            if (user, movie) in self.predicted:
                estimated_rating = self.predicted[user, movie]
                details = {'was_impossible' : False}
            else:
                estimated_rating = self.mean_rating
                details = {'was_impossible' : True}
            predictions.append(Prediction(uid = user,
                                          iid = movie,
                                          r_ui = rating,
                                          est = estimated_rating,
                                          details = details))

        return predictions
    
    def __sharp_push__(self, G, node, node_to_idx, p, r, prob, eps):
        u = node_to_idx[node]

        upd = r[u] / ((2.0 + self.beta) * G.degree[node])
        for node_v in G.neighbors(node):
            v = node_to_idx[node_v]
            r[v] = r[v] + upd
            if r[v] > eps * G.degree[node_v]:
                prob.add(node_v)

        p[u] = p[u] + self.beta / (2.0 + self.beta) * r[u]
        r[u] = r[u] / (2.0 + self.beta)

        if r[u] <= eps * G.degree[node]:
            prob.remove(node)

        return p, r, prob

    def __sharp_approximate__(self, G, node_to_idx, s, eps):
        p = np.zeros(len(G))
        r = s

        prob = set([])
        for node in G.nodes:
            if r[node_to_idx[node]] > eps * G.degree[node]:
                prob.add(node)

        while prob:
            node = next(iter(prob))
            p, r, prob = self.__sharp_push__(G, node, node_to_idx, p, r, prob, eps)

        return p, r

    def __sharp_pagerank__(self, G, node_to_idx, t = None):
        n = len(G)

        if t is None:
            s = np.ones(n) / n
        else:
            s = np.zeros(n)
            for node, tw in t.items():
                s[node_to_idx[node]] = tw

        c_eps = 1.0
        p = np.zeros(n)
        r = s

        while c_eps > self.eps:
            c_eps /= 2
            p_d, r_d = self.__sharp_approximate__(G, node_to_idx, r, c_eps)
            p += p_d
            r = r_d

        ret = {node : p[node_to_idx[node]] for node in G.nodes}
        return ret

In [3]:
data = Dataset.load_builtin('ml-100k')
train_data, test_data = model_selection.train_test_split(data, test_size = 0.2)

In [4]:
pr = PageRankRecommender(alpha = 0.7, eps = 1e-6)
pr.fit(train_data, val_data = test_data)

In [5]:
pr_predictions = pr.test(test_data)
accuracy.rmse(pr_predictions)
accuracy.mae(pr_predictions)

RMSE: 0.9729
MAE:  0.7687


0.7687455606101018

In [6]:
svd = SVD(n_epochs = 10, lr_all = 0.005, reg_all = 0.4)
svd.fit(train_data)

In [7]:
predictions = svd.test(test_data)
accuracy.rmse(predictions)
accuracy.mae(predictions)

RMSE: 0.9585
MAE:  0.7680


0.7680454477474217